# Eigenvalue extraction during simulation of a single-phase RLC circuit.

- Simulate RLC circuit in DP domain.
- Simulation includes switch events changing resistance value of the circuit.
- Extract eigenvalues at each time step of simulation.
- Visualize eigenvalue trajectory.

In [ ]:
import dpsimpy
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt

In [ ]:
# DPsim DP simulation
name = 'DP_SinglePhaseRLC_Demo'
log_dir = "logs/" + name
dpsimpy.Logger.set_log_dir(log_dir)

# Create nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n2 = dpsimpy.dp.SimNode('n2')
n3 = dpsimpy.dp.SimNode('n3')
n4 = dpsimpy.dp.SimNode('n4')
n5 = dpsimpy.dp.SimNode('n5')

# Create components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(100,0))
r1 = dpsimpy.dp.ph1.Resistor('r_1')
r1.set_parameters(R=10)
r2 = dpsimpy.dp.ph1.Resistor('r_2')
r2.set_parameters(R=0.1)
l = dpsimpy.dp.ph1.Inductor('l')
l.set_parameters(L=5)
c = dpsimpy.dp.ph1.Capacitor('c')
c.set_parameters(C=250e-6)
s1 = dpsimpy.dp.ph1.Switch('s1')
s1.set_parameters(1e8, 1e-4, True)
s2 = dpsimpy.dp.ph1.Switch('s2')
s2.set_parameters(1e8, 1e-4, False)

# Create switch events
s1Event = dpsimpy.event.SwitchEvent(0.05, s1, False)
s2Event = dpsimpy.event.SwitchEvent(0.05, s2, True)

# Set connections
vs.connect([gnd, n1])
r1.connect([n4, gnd])
r2.connect([n5, gnd])
l.connect([n1, n2])
c.connect([n2, n3])
s1.connect([n3, n4])
s2.connect([n3, n5])

# Create topology
system = dpsimpy.SystemTopology(50, [n1, n2, n3, n4, n5], [vs, r1, r2, l, c, s1, s2])

# Configure logging
logger = dpsimpy.Logger(name)
logger.log_attribute('current', 'i_intf', l)
logger.log_attribute('voltage', 'v', n3)

# Configure and run simulation
sim = dpsimpy.Simulation(name)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_system(system)
sim.add_event(s1Event)
sim.add_event(s2Event)
sim.do_eigenvalue_extraction(True)
sim.set_time_step(0.0001)
sim.set_final_time(0.1)
sim.add_logger(logger)
sim.run()

# Read log file
eigenvalues_timeseries = rt.read_timeseries_dpsim(log_dir + '/eigenvalues.csv')
time = eigenvalues_timeseries['eigenvalues_0'].time
eigenvalue1_real = eigenvalues_timeseries['eigenvalues_0'].values.real
eigenvalue1_imag = eigenvalues_timeseries['eigenvalues_0'].values.imag
eigenvalue2_real = eigenvalues_timeseries['eigenvalues_1'].values.real
eigenvalue2_imag = eigenvalues_timeseries['eigenvalues_1'].values.imag

simulation_timeseries = rt.read_timeseries_dpsim(log_dir + '/' + name + '.csv')
simulation_timeseries_time_domain = ts.frequency_shift_list(simulation_timeseries, 50)
print(simulation_timeseries_time_domain)
current_timeseries = simulation_timeseries_time_domain['current_shift']
voltage_timeseries = simulation_timeseries_time_domain['voltage_shift']

In [ ]:
plt.figure()
plt.title('Current')
plt.plot(current_timeseries.time, current_timeseries.values)
plt.show()

plt.figure()
plt.title('Voltage across resistor')
plt.plot(voltage_timeseries.time, voltage_timeseries.values)
plt.show()

plt.figure()
plt.title('First eigenvalue')
plt.plot(time, eigenvalue1_real, label='Real part')
plt.plot(time, eigenvalue1_imag, label='Imaginary part')
plt.legend()
plt.show()

plt.figure()
plt.title('Second eigenvalue')
plt.plot(time, eigenvalue2_real, label='Real part')
plt.plot(time, eigenvalue2_imag, label='Imaginary part')
plt.legend()
plt.show()